---

## Ejercicio 4: Desafío Autónomo - Análisis de Reseñas de Restaurantes

### Contexto del problema

Sos el encargado de marketing digital de una cadena de restaurantes porteña. Querés implementar un sistema inteligente que procese automáticamente las reseñas que los clientes dejan en Google Maps y redes sociales para:

1. Identificar si la reseña es positiva, negativa o neutral
2. Extraer información clave: nombres de platos mencionados, ubicaciones de las sucursales, nombres de empleados destacados
3. Responder automáticamente a preguntas frecuentes basándose en el menú y políticas del restaurante

### Aplicación real

Este tipo de sistemas combinados se usan en:
- Gestión de reputación online para cadenas de restaurantes
- Análisis de feedback de clientes en hotelería y turismo
- Sistemas de CRM (Customer Relationship Management) inteligentes
- Plataformas de delivery con análisis de satisfacción del cliente

---

### Tu tarea

**Este ejercicio lo tenés que resolver completamente solo**, aplicando todo lo que aprendiste en los ejercicios anteriores. No hay código de ejemplo, solo las instrucciones.

### Parte 1: Análisis de sentimiento de reseñas (30%)

1. Creá una lista con al menos 5 reseñas ficticias de clientes sobre un restaurante argentino (podés inventarlas o buscar reales)
2. Cargá un modelo de análisis de sentimientos en español (buscá en Hugging Face)
3. Clasificá cada reseña y mostrá los resultados en un DataFrame
4. Identificá cuántas reseñas son positivas, negativas y neutrales (si el modelo lo soporta)

**Pistas:**
- Usá `pipeline("text-classification", model=...)`
- Recordá importar `pandas` para crear el DataFrame
- Modelos sugeridos: `finiteautomata/beto-sentiment-analysis` o `pysentimiento/robertuito-sentiment-analysis`

---

### Parte 2: Extracción de información (40%)

1. Tomá 2 de las reseñas que creaste (las más largas y detalladas)
2. Cargá un modelo de NER en español
3. Extraé todas las entidades nombradas de esas reseñas
4. Organizá la información en categorías (personas, lugares, organizaciones)
5. Bonus: ¿Se mencionan nombres de platos? (Nota: el modelo podría no detectarlos como entidades, reflexioná sobre por qué)

**Pistas:**
- Usá `pipeline("ner", model=..., aggregation_strategy="simple")`
- Modelo sugerido: `mrm8488/bert-spanish-cased-finetuned-ner`
- Recordá iterar sobre los resultados para organizarlos por tipo

---

### Parte 3: Sistema de preguntas y respuestas (30%)

1. Escribí un texto con información del restaurante (menú, horarios, ubicación, políticas de reservas, precios promedio, etc.). Mínimo 4-5 oraciones.
2. Cargá un modelo de Question Answering en español
3. Formulá al menos 4 preguntas que un cliente podría hacer
4. Generá respuestas automáticas usando el modelo
5. Mostrá cada pregunta con su respuesta y el nivel de confianza del modelo

**Pistas:**
- Usá `pipeline("question-answering", model=...)`
- Modelo sugerido: `PlanTL-GOB-ES/roberta-base-bne-sqac`
- La función necesita dos parámetros: `question=` y `context=`

---

### Bonus (opcional): Integración completa

Si terminaste las tres partes, intentá crear una función que:
1. Reciba una reseña de cliente como input
2. Analice el sentimiento
3. Extraiga entidades mencionadas
4. Genere un resumen estructurado

Por ejemplo:
```
RESEÑA: "Fui ayer a la sucursal de Palermo y el mozo Juan me atendió bárbaro..."

ANÁLISIS:
- Sentimiento: POSITIVO (95% confianza)
- Empleado mencionado: Juan
- Sucursal: Palermo
- Recomendación: Enviar agradecimiento personalizado
```

---

### Criterios de evaluación

Evaluá tu propio trabajo considerando:

1. **Funcionalidad (50%):** ¿El código funciona sin errores? ¿Completaste las tres partes?
2. **Calidad de datos (20%):** ¿Las reseñas y preguntas son realistas? ¿El contexto tiene información útil?
3. **Presentación (20%):** ¿Los resultados se muestran de forma clara? ¿Usaste DataFrames o print statements organizados?
4. **Reflexión crítica (10%):** ¿Analizaste la calidad de las predicciones? ¿Identificaste limitaciones?

---

### Espacio para tu solución

Usá las celdas siguientes para resolver el desafío. Podés crear todas las celdas que necesites.


In [1]:
# PARTE 1: ANÁLISIS DE SENTIMIENTO

!pip install transformers torch pandas -q

from transformers import pipeline

import pandas as pd

comentarios = [
    "La carta es amplia y con mucha variedad de platos, pero las pastas son un must. Vale la pena probar el risotto de osobuco. Los platos son abundantes, para compartir idealmente. Esta vez fuimos en grupo, 10 personas y todos comimos excelente gracias a la impecable atención de Dani “el Pelado”. Dani tuvo en cuenta nuestras preferencias y fue súper amable durante toda la noche. La comida llegó perfecta. De postre pedimos tiramisú y flan. El tiramisú es sencillo, sabroso y suave. Volveremos!",
    "Reserve por WhatsApp el mismo día , hay dos turnos me tocó es de las 22 hs , esperamos 20 min e ingresamos , nos recibieron con un pan de pizza , muy rico, los platos son para compartir si o si , pedimos langostinos y calamaretis de entrada y principal ravioles de ternera al vino tinto con salsa carbonara y risotto de osobuco. Fuimos cuatro , y quedamos sumamente satisfechos , primera vez pero no la última.",
    "Hola! La voy a hacer cortita y al pie, pedimos fideos negros con salsa frutos de mar que nos pareció demasiada cara para lo que era, de postre volcán de chocolate que estaba pasado, y lo peor no fue eso, sino que al momento de pagar la cuenta por posnet nos preguntó si queríamos dejar propina por efectivo o tarjeta, le dijimos por tarjeta y monto que elegiamos para la propina pero el mozo terminó eligiendo el sin consultarnos y dijo “mínimo %10”  nos pareció de mala educación, por todo eso le pongo un 1.",
    "El trato no me gustó para nada, desde que llegué temprano el trato fue malo, la comida abundante pero los fideos no estaban al dente, tampoco acorde a la expectativa, comimos bien, nada más, no volvería, lo único destacado, la excelente cocción del marisco, me pareció más pretensioso que excelente!",
    "Lamentablemente me quedo con una mala experiencia y por eso modificó mi calificación que hace 2 años fue 5 Estrellas. Los precios de los platos son desproporcionados por no decir un DELIRIO, todas las pastas con las salsas desde 33.000 pesos, chipirones 35.000, milanesa pamigina 20,000, pulpo 160.000 pesos, no hablar de las entradas … todo esto en un lugar MUY DESCUIDADO con los techos rotos, pedazo de pintura colgando … si queres tener el mejor italiano d Buenos Aires no podes tener esta infraestructura en malas condiciones … me llevo la decepción, ya que los papardelle con su salsa bolognesa era muy malos … o sea bajo la calidad de comida y los precios son ULTRA CAROS y más q está en parque Patricios donde el costo del alquiler debería ser más bajo que en palermo, recoleta o similar … no creo volver relación Calidad / Precio, muy malo una lástima se les fue la fama al humo y olvidaron de cuidar la calidad y costo de precios",
    "Fuimos a comer con mis compañeros de trabajo al mediodia, la atención fue muy buena, pero la comida (si bien muchos platos son para compartir) era normal... Los precios son elevados para la calidad de la comida. Considero que solo el risotto es un plato considerable para compartir. Los demás (según el apetito de cada comenzal) son para una sola persona (como por ejemplo los ravioles) y siendo para una sola persona el valor es elevado."
]

# Cargamos un modelo de análisis de sentimientos específico para español
clasificador = pipeline(
    "text-classification",
    model="finiteautomata/beto-sentiment-analysis"
)

print("Modelo cargado correctamente.")

# Procesamos los comentarios
resultados = clasificador(comentarios)

# Creamos un DataFrame para visualizar los resultados
df_resultados = pd.DataFrame({
    'Comentario': comentarios,
    'Sentimiento': [r['label'] for r in resultados],
    'Confianza': [round(r['score'], 3) for r in resultados]
})

print("="*30)
# Mostramos Dataframe
print(df_resultados)
print("\n")
print("="*30)

# Contamos cuantas reseñas hay por tipo de sentimiento
conteo_sentimientos = df_resultados['Sentimiento'].value_counts()

# Mostramos los resultados
print("Cantidad de reseñas por tipo de sentimiento:")
print(conteo_sentimientos)
print("\n")
print("="*30)

# Mostramos los resultados con porcentajes
porcentaje_sentimientos = df_resultados['Sentimiento'].value_counts(normalize=True) * 100
print("\nPorcentaje de cada tipo de sentimiento:")
print(porcentaje_sentimientos.round(2))



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu


Modelo cargado correctamente.
                                          Comentario Sentimiento  Confianza
0  La carta es amplia y con mucha variedad de pla...         POS      0.999
1  Reserve por WhatsApp el mismo día , hay dos tu...         POS      0.999
2  Hola! La voy a hacer cortita y al pie, pedimos...         NEG      0.996
3  El trato no me gustó para nada, desde que lleg...         POS      0.990
4  Lamentablemente me quedo con una mala experien...         NEG      0.999
5  Fuimos a comer con mis compañeros de trabajo a...         NEU      0.963


Cantidad de reseñas por tipo de sentimiento:
Sentimiento
POS    3
NEG    2
NEU    1
Name: count, dtype: int64



Porcentaje de cada tipo de sentimiento:
Sentimiento
POS    50.00
NEG    33.33
NEU    16.67
Name: proportion, dtype: float64


In [2]:
# PARTE 2: EXTRACCIÓN DE INFORMACIÓN (NER)

# Cargamos las 2 reseñas más largas/detalladas
comentarios_ner = [
    "La carta es amplia y con mucha variedad de platos, pero las pastas son un must. Vale la pena probar el risotto de osobuco. Los platos son abundantes, para compartir idealmente. Esta vez fuimos en grupo, 10 personas y todos comimos excelente gracias a la impecable atención de Dani El Pelado. Dani tuvo en cuenta nuestras preferencias y fue súper amable durante toda la noche. La comida llegó perfecta. De postre pedimos tiramisú y flan. El tiramisú es sencillo, sabroso y suave. Volveremos!",
    "Lamentablemente me quedo con una mala experiencia y por eso modificó mi calificación que hace 2 años fue 5 Estrellas. Los precios de los platos son desproporcionados por no decir un DELIRIO, todas las pastas con las salsas desde 33.000 pesos, chipirones 35.000, milanesa pamigina 20,000, pulpo 160.000 pesos, no hablar de las entradas … todo esto en un lugar MUY DESCUIDADO con los techos rotos, pedazo de pintura colgando … si queres tener el mejor italiano d Buenos Aires no podes tener esta infraestructura en malas condiciones … me llevo la decepción, ya que los papardelle con su salsa bolognesa era muy malos … o sea bajo la calidad de comida y los precios son ULTRA CAROS y más q está en Parque Patricios donde el costo del alquiler debería ser más bajo que en Palermo, Recoleta o similar … no creo volver relación Calidad / Precio, muy malo una lástima se les fue la fama al humo y olvidaron de cuidar la calidad y costo de precios"]

# Cargamos un modelo de NER específico para español
extractor_ner = pipeline(
    "ner",
    model="mrm8488/bert-spanish-cased-finetuned-ner",
    aggregation_strategy="simple"
)

print("\nModelo NER cargado correctamente.")

# Procesamos comentarios
entidades = extractor_ner(comentarios_ner)
entidades = [e for lista in entidades for e in lista]

# Creamos un DataFrame con entidades
df_entidades = pd.DataFrame(entidades)

print("\nMostramos dataframe generado")
print("="*60)
print(df_entidades)
print("="*60)

# Mostramos solo las columnas relevantes

df_entidades[['entity_group', 'word', 'score']].round(3)

# Diccionario para organizar por tipo de entidad
info_extraida = {
    'PER': [],  # Personas
    'ORG': [],  # Organizaciones
    'LOC': []   # Ubicaciones
}

# Clasificamos cada entidad
for entidad in entidades:
    tipo = entidad['entity_group']
    palabra = entidad['word']
    if tipo in info_extraida:
        info_extraida[tipo].append(palabra)

# Mostramos la información organizada
print("="*60)
print("INFORMACIÓN EXTRAÍDA DE LOS COMENTARIOS")
print("="*60)
print(f"\nPersonas: {', '.join(info_extraida['PER'])}")
print(f"\nOrganizaciones: {', '.join(info_extraida['ORG'])}")
print(f"\nLugares: {', '.join(info_extraida['LOC'])}")
print("="*60)
print("="*60)
print("Observaciones")
print("El modelo no pudo detectar los nombres de platos como entidades ya que seguramente no esté entrenado con un corpus de lenguaje gastronómico.")
print("Palabras como 'Risotto' o 'Tiramisú' son solo palabras para el modelo utilizado, no puede clasificarlas bajo ninguna etiqueta.")
print("Haciendo una analogía, sería como pedirle a un modelo entrenado para detectar 'marcas de autos' que nos identifique 'marcas de perfumes'. No lo va a hacer, no porque el texto esté mal, sino porque no aprendió ese concepto.")
print("="*60)


Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



Modelo NER cargado correctamente.

Mostramos dataframe generado
  entity_group     score              word  start  end
0          PER  0.999684               Dan    276  279
1          PER  0.999351     ##i El Pelado    279  290
2          PER  0.999550               Dan    292  295
3          PER  0.989620               ##i    295  296
4         MISC  0.586076               DEL    182  185
5          LOC  0.999419      Buenos Aires    461  473
6          LOC  0.999383  Parque Patricios    695  711
7          LOC  0.999827           Palermo    768  775
8          LOC  0.981946          Recoleta    777  785
9         MISC  0.925662           Calidad    822  829
INFORMACIÓN EXTRAÍDA DE LOS COMENTARIOS

Personas: Dan, ##i El Pelado, Dan, ##i

Organizaciones: 

Lugares: Buenos Aires, Parque Patricios, Palermo, Recoleta
Observaciones
El modelo no pudo detectar los nombres de platos como entidades ya que seguramente no esté entrenado con un corpus de lenguaje gastronómico.
Palabras como 'Ri

In [3]:
# PARTE 3: QUESTION ANSWERING
politicas_ilombu = """IL OMBU TRATTORIA ITALIANA – POLÍTICAS, INFORMACIÓN Y DATOS DE SERVICIO

1. Información general
IL OMBU Trattoria Italiana está ubicado en Patagones 2976, Parque Patricios, Ciudad de Buenos Aires (C.P. 1437).
Ofrece una propuesta gastronómica de estilo trattoria moderna, con especialidad en pastas artesanales, risottos, pizzas de masa fina, platos para compartir, cortes de carne y postres clásicos.

2. Horarios de atención
Abierto de lunes a domingos entre 20:00 hs y 01:00 hs.
Se recomienda reserva previa, especialmente fines de semana y fechas especiales.

3. Menú y rango de precios
El menú incluye:
– Pastas artesanales (fettuccine, pappardelle, ravioles rellenos, etc.)
– Risottos (incluye opciones de osobuco)
– Pizzas finas estilo trattoria
– Entradas para compartir (pan de pizza especial, bruschettas, etc.)
– Cortes argentinos seleccionados
– Postres clásicos italianos (tiramisú, flan, etc.)

El consumo promedio por persona, incluyendo plato principal y postre (sin bebidas), ronda entre los $15.000 y $20.000 según la elección del menú vigente.

4. Política de reservas
– Las reservas se gestionan por WhatsApp o teléfono.
– Para grupos de 6 o más puede solicitarse confirmación con tarjeta o anticipo.
– Para respetar los tiempos del salón, los turnos cuentan con tolerancia de llegada.
– En caso de cancelación, se solicita aviso con antelación.

5. Ubicación y acceso
Situado en Parque Patricios, con acceso por transporte público y estacionamiento en la vía pública o cocheras cercanas.
El ambiente es moderno, cálido y apto tanto para cenas en pareja como en grupo.

6. Precios y formas de pago
– Precios sujetos a actualización de carta vigente.
– Se aceptan tarjetas de crédito/débito, pagos digitales y efectivo.
– Bebidas y servicio de mesa se abonan según carta vigente.
– Para platos compartidos puede aplicarse cobertura cuando corresponda.

7. Política de cancelaciones / modificaciones
– Modificaciones de horario o cantidad de comensales deben informarse con 12 horas de antelación.
– La reserva puede liberarse ante demoras sin aviso.
– Para eventos o grupos grandes pueden aplicarse condiciones especiales de menú o consumo mínimo.

8. Recomendaciones de experiencia
– Entradas para compartir (pan de pizza de la casa, bruschettas).
– Platos destacados para dos: risotto de osobuco o pastas artesanales de autor.
– Postres recomendados: tiramisú o flan clásico.
– Se sugiere llegar entre 20:00 y 21:00 para mejor disponibilidad de mesas.
– Ideal para grupos de 4 a 10 y también para cenas íntimas en pareja."""

# Cargamos modelo de Question Answering
qa_modelo = pipeline(
    "question-answering",
    #model="PlanTL-GOB-ES/roberta-base-bne-sqac"
    model="mrm8488/bert-multi-uncased-finetuned-xquadv1",
    tokenizer="mrm8488/bert-multi-uncased-finetuned-xquadv1"
)

print("Modelo de QA cargado correctamente.")
print("Utilizamos el modelo extraído de 'https://huggingface.co/mrm8488/bert-multi-uncased-finetuned-xquadv1' ya que el recomendado solicita token de acceso.")

# Definimos prreguntas típicas de clientes
preguntas = [
    "¿Cuál es su dirección?",
    "¿Cuál es su horario de atención?",
    "¿Cómo puedo realizar una reserva?",
    "¿Qué platos recomiendan para dos personas?",
    "¿Tienen estacionamiento?"
]

# Procesamos cada prregunta
print("RESPUESTAS AUTOMÁTICAS DEL CHATBOT")
print("="*60)

for pregunta in preguntas:
    respuesta = qa_modelo(question=pregunta, context=politicas_ilombu)
    print(f"\nPregunta: {pregunta}")
    print(f"Respuesta: {respuesta['answer']}")
    print(f"Confianza: {respuesta['score']*100:.1f}%")
    print("-"*60)

Some weights of the model checkpoint at mrm8488/bert-multi-uncased-finetuned-xquadv1 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


Modelo de QA cargado correctamente.
Utilizamos el modelo extraído de 'https://huggingface.co/mrm8488/bert-multi-uncased-finetuned-xquadv1' ya que el recomendado solicita token de acceso.
RESPUESTAS AUTOMÁTICAS DEL CHATBOT

Pregunta: ¿Cuál es su dirección?
Respuesta: Patagones 2976
Confianza: 25.9%
------------------------------------------------------------

Pregunta: ¿Cuál es su horario de atención?
Respuesta: de lunes a domingos entre 20:00 hs y 01:00 hs
Confianza: 19.8%
------------------------------------------------------------

Pregunta: ¿Cómo puedo realizar una reserva?
Respuesta: WhatsApp o teléfono
Confianza: 67.9%
------------------------------------------------------------

Pregunta: ¿Qué platos recomiendan para dos personas?
Respuesta: risotto de osobuco o pastas artesanales de autor
Confianza: 107.5%
------------------------------------------------------------

Pregunta: ¿Tienen estacionamiento?
Respuesta: vía pública o cocheras cercanas
Confianza: 116.2%
-----------------

In [4]:
# BONUS (OPCIONAL): INTEGRACIÓN COMPLETA
from transformers import pipeline
import pandas as pd

# Cargar modelos una sola vez
clasificador_sent = pipeline(
    "text-classification",
    model="finiteautomata/beto-sentiment-analysis"
)
clasificador_ner = pipeline(
    "ner",
    model="mrm8488/bert-spanish-cased-finetuned-ner",
    aggregation_strategy="simple"
)

def analizar_reseña(reseña):
    """
    Analiza una reseña de cliente:
    - Detecta sentimiento
    - Extrae entidades (personas, organizaciones, ubicaciones)
    - Genera resumen estructurado
    """

    # Analizamos sentimiento
    resultado_sent = clasificador_sent(reseña)[0]
    sentimiento = resultado_sent['label']
    confianza_sent = round(resultado_sent['score'], 3) * 100

    # Extraemos entidades
    entidades = clasificador_ner(reseña)
    info = {'PER': [], 'ORG': [], 'LOC': []}

    for e in entidades:
        if e['entity_group'] in info:
            info[e['entity_group']].append(e['word'])

    # Acciones a tomar acorde al sentimiento detectado
    recomendacion = ""
    if sentimiento == "POS":
        recomendacion += "Enviar agradecimiento personalizado."
    elif sentimiento == "NEG":
        recomendacion += "Revisar situación y contactar cliente."
    else:
        recomendacion += "No se requiere acción inmediata."

    # Contruimos resumen de reseña como un objeto
    resumen = {
        "RESEÑA": reseña,
        "ANÁLISIS": {
            "Sentimiento": f"{sentimiento} ({confianza_sent:.1f}% confianza)",
            "Persona/s": ', '.join(info['PER']) if info['PER'] else None,
            "Lugar/es": ', '.join(info['LOC']) if info['LOC'] else None,
            "Organización": ', '.join(info['ORG']) if info['ORG'] else None,
            "Recomendación": recomendacion
        }
    }

    return resumen

Device set to use cpu
Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [5]:
resultado = analizar_reseña("Excelente todo! Nos atendió Juan, un genio. La cercanía con el subte es lo mejor, a metros del Parque Patricios.")
print("="*100)
print(f"Reseña: {resultado['RESEÑA']}\n")
print("="*60)
print(f"Sentimiento: {resultado['ANÁLISIS']['Sentimiento']}")
print(f"Personas mencionadas: {resultado['ANÁLISIS']['Persona/s']}")
print(f"Organizaciones mencionadas: {resultado['ANÁLISIS']['Organización']}")
print(f"Lugares mencionados: {resultado['ANÁLISIS']['Lugar/es']}\n")
print(f"Acciones a tomar: {resultado['ANÁLISIS']['Recomendación']}")
print("="*100)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Reseña: Excelente todo! Nos atendió Juan, un genio. La cercanía con el subte es lo mejor, a metros del Parque Patricios.

Sentimiento: POS (99.9% confianza)
Personas mencionadas: Juan
Organizaciones mencionadas: None
Lugares mencionados: Parque Patricios

Acciones a tomar: Enviar agradecimiento personalizado.


In [6]:
resultado = analizar_reseña("Pésimo servicio, Juan es un maltratador que no debería estar atendiendo personas. De los peores lugares para comer en Buenos Aires.")
print("="*100)
print(f"Reseña: {resultado['RESEÑA']}\n")
print("="*60)
print(f"Sentimiento: {resultado['ANÁLISIS']['Sentimiento']}")
print(f"Personas mencionadas: {resultado['ANÁLISIS']['Persona/s']}")
print(f"Organizaciones mencionadas: {resultado['ANÁLISIS']['Organización']}")
print(f"Lugares mencionados: {resultado['ANÁLISIS']['Lugar/es']}\n")
print(f"Acciones a tomar: {resultado['ANÁLISIS']['Recomendación']}")
print("="*100)

Reseña: Pésimo servicio, Juan es un maltratador que no debería estar atendiendo personas. De los peores lugares para comer en Buenos Aires.

Sentimiento: NEG (99.9% confianza)
Personas mencionadas: Juan
Organizaciones mencionadas: None
Lugares mencionados: Buenos Aires

Acciones a tomar: Revisar situación y contactar cliente.


In [7]:
resultado = analizar_reseña("Buen lugar, con sus pro y sus contras. Es rico, pero tiene precios excesivos. Deberían avisar que los platos son para compartir, cuando pedímos, Juan no nos informó esto y terminó sobrando mucha comida.")
print("="*100)
print(f"Reseña: {resultado['RESEÑA']}\n")
print("="*60)
print(f"Sentimiento: {resultado['ANÁLISIS']['Sentimiento']}")
print(f"Personas mencionadas: {resultado['ANÁLISIS']['Persona/s']}")
print(f"Organizaciones mencionadas: {resultado['ANÁLISIS']['Organización']}")
print(f"Lugares mencionados: {resultado['ANÁLISIS']['Lugar/es']}\n")
print(f"Acciones a tomar: {resultado['ANÁLISIS']['Recomendación']}")
print("="*100)

Reseña: Buen lugar, con sus pro y sus contras. Es rico, pero tiene precios excesivos. Deberían avisar que los platos son para compartir, cuando pedímos, Juan no nos informó esto y terminó sobrando mucha comida.

Sentimiento: NEU (89.4% confianza)
Personas mencionadas: Juan
Organizaciones mencionadas: None
Lugares mencionados: None

Acciones a tomar: No se requiere acción inmediata.


### Reflexión personal

Después de completar el ejercicio, respondé estas preguntas:

1. **¿Qué fue lo más difícil del ejercicio?**
   - Me encontré con dos problemas: 1. No terminaba de comprender porqué se presentaban fallas en el modelo de Q&A, pero investigando vi que el modelo recomendado requiere token. Esto me llevo a investigar en huggingface hasta dar con un modelo compatible con el uso que iba a darle. 2. Al final, en el punto opcional, no lograba generar una reseña que me clasifique como NEUTRAL. Creo haber comprendido que un sentimiento "Neutral" debería tener tantas palabras "positivas" como "negativas", supongo que algunas tendrán más peso que otras, pero más o menos esa fue la traba al aplicar "ingeniería inversa": ir poniendo reseñas que uno creería 'Neutrales' hasta que el sistema las detecte como tal.

2. **¿Encontraste alguna limitación en los modelos?**
   - Los modelos son bastante "genéricos" o estaban entrenados para otro tipo de uso (no específicos de gastronomía). También encuentra dificultades para clasificar personas en el NER.

3. **¿Cómo podrías mejorar este sistema para un caso real?**
   - Buscar y/o entrenar modelos específicos para el caso de uso, en el trabajo sería ideal hacer uso de modelos entrenados con corpus gastronómicos por ejemplo.

4. **¿Qué otras aplicaciones se te ocurren para estas técnicas en el contexto argentino?**
   - Análisis de sentimiento en conversaciones con objetivos de control de calidad (sistema de atención al cliente por Whatsapp, por ejemplo). Bot de respuestas automáticas a valoraciones o comentarios en redes sociales, modificando la respuesta en base al sentimiento y entidades mencionadas.

---

---

## Reflexiones Finales del Curso

Completaste los cuatro ejercicios de NLP con Transformers, incluyendo un desafío autónomo. Vamos a reflexionar sobre lo aprendido:

### Ejercicio 1 - Moderación de Comentarios
- **Aprendiste:** A clasificar texto usando modelos de análisis de sentimientos
- **Aplicación:** Moderación automática de redes sociales, priorización de tickets de soporte
- **Limitaciones:** Los modelos pueden tener problemas con ironía, sarcasmo o lenguaje muy coloquial

### Ejercicio 2 - Extracción de CVs
- **Aprendiste:** A extraer entidades nombradas (personas, organizaciones, lugares) de texto
- **Aplicación:** Automatización de RRHH, análisis de documentos, extracción de información estructurada
- **Limitaciones:** Algunos modelos pueden confundir entidades o no detectar nombres poco comunes

### Ejercicio 3 - Chatbot de Soporte
- **Aprendiste:** A usar Question Answering para responder preguntas basadas en un contexto
- **Aplicación:** Chatbots, asistentes virtuales, sistemas de FAQ automáticas
- **Limitaciones:** El modelo solo puede responder sobre información presente en el contexto

### Ejercicio 4 - Desafío Autónomo
- **Aprendiste:** A combinar múltiples técnicas de NLP para resolver un problema real completo
- **Aplicación:** Sistemas integrados de análisis de feedback, gestión de reputación online
- **Habilidad clave:** Autonomía para investigar, implementar y evaluar soluciones de NLP

### Próximos pasos sugeridos

1. **Explorá más modelos** en [Hugging Face Hub](https://huggingface.co/models)
2. **Combiná técnicas:** Por ejemplo, usá clasificación de sentimientos + QA para un chatbot más inteligente
3. **Experimentá con otros idiomas** o dialectos regionales
4. **Investigá fine-tuning:** Aprendé a ajustar modelos con tus propios datos
5. **Desarrollá un proyecto propio:** Elegí un problema real que te interese y aplicá estas técnicas

---

## Glosario Técnico

### Conceptos fundamentales

**Transformer**  
Arquitectura de red neuronal basada en mecanismos de atención que revolucionó el NLP en 2017. Permite procesar secuencias de texto completas simultáneamente en lugar de palabra por palabra.

**Pipeline**  
Interfaz de alto nivel en Hugging Face que encapsula todo el proceso de preprocesamiento, inferencia y postprocesamiento de un modelo. Facilita el uso de modelos preentrenados con pocas líneas de código.

**Modelo preentrenado**  
Modelo de machine learning que fue entrenado previamente con grandes cantidades de datos. Puede usarse directamente o ajustarse (fine-tuning) para tareas específicas.

**Tokenización**  
Proceso de dividir texto en unidades más pequeñas (tokens) que el modelo puede procesar. Puede ser a nivel de palabras, subpalabras o caracteres.

### Tareas de NLP

**Text Classification (Clasificación de texto)**  
Tarea de asignar una o más etiquetas a un texto. Incluye análisis de sentimientos, detección de spam, clasificación de temas, etc.

**Sentiment Analysis (Análisis de sentimientos)**  
Subtipo de clasificación que identifica la polaridad emocional de un texto (positivo, negativo, neutral). Se usa en redes sociales, reseñas de productos, atención al cliente.

**Named Entity Recognition - NER (Reconocimiento de entidades nombradas)**  
Tarea de identificar y clasificar nombres propios en texto: personas (PER), organizaciones (ORG), ubicaciones (LOC), fechas, cantidades monetarias, etc.

**Question Answering - QA (Respuesta a preguntas)**  
Tarea de responder preguntas en lenguaje natural basándose en un contexto dado. El modelo extrae la respuesta directamente del texto proporcionado.

**Text Generation (Generación de texto)**  
Tarea de crear texto nuevo de manera coherente a partir de un prompt inicial. Incluye completado de texto, escritura creativa, chatbots conversacionales.

**Summarization (Resumen automático)**  
Tarea de condensar un texto largo en una versión más corta manteniendo la información más importante.

**Translation (Traducción automática)**  
Tarea de traducir texto de un idioma a otro usando modelos de secuencia a secuencia.

### Componentes técnicos

**Embedding (Representación vectorial)**  
Representación numérica de palabras o tokens como vectores en un espacio multidimensional. Palabras con significados similares tienen embeddings cercanos.

**Attention (Atención)**  
Mecanismo que permite al modelo enfocarse en diferentes partes del input al procesar cada elemento. Es el componente clave de la arquitectura Transformer.

**Fine-tuning (Ajuste fino)**  
Proceso de tomar un modelo preentrenado y entrenarlo adicionalmente con datos específicos de tu dominio para mejorar su desempeño en tu tarea particular.

**Inference (Inferencia)**  
Proceso de usar un modelo ya entrenado para hacer predicciones sobre datos nuevos. No implica entrenamiento, solo aplicación del modelo.

**Score / Confidence (Puntuación / Confianza)**  
Valor numérico (generalmente entre 0 y 1) que indica qué tan seguro está el modelo de su predicción. Valores más altos indican mayor confianza.

### Modelos mencionados

**BETO**  
Versión de BERT (Bidirectional Encoder Representations from Transformers) entrenada específicamente con texto en español. Usado en análisis de sentimientos y otras tareas de clasificación.

**RoBERTa**  
Variante optimizada de BERT con mejoras en el proceso de preentrenamiento. Usado para múltiples tareas de NLP en español.

**BERT (Bidirectional Encoder Representations from Transformers)**  
Modelo Transformer que lee texto bidireccionalmente (izquierda a derecha y derecha a izquierda simultáneamente) para comprender mejor el contexto.

**GPT (Generative Pre-trained Transformer)**  
Familia de modelos diseñados específicamente para generación de texto. Leen texto de izquierda a derecha y predicen la siguiente palabra.

### Plataformas y librerías

**Hugging Face**  
Plataforma y empresa que desarrolla herramientas de NLP de código abierto. Su librería Transformers es el estándar de facto para trabajar con modelos de lenguaje.

**Hugging Face Hub**  
Repositorio online con miles de modelos preentrenados, datasets y demos interactivas. Permite compartir y descargar modelos fácilmente.

**PyTorch / TensorFlow**  
Frameworks de deep learning usados como backend por la librería Transformers. PyTorch es más común en investigación, TensorFlow en producción.

### Métricas y evaluación

**Label (Etiqueta)**  
Categoría asignada por el modelo a un texto. En análisis de sentimientos: POS (positivo), NEG (negativo), NEU (neutral).

**Aggregation strategy (Estrategia de agregación)**  
En NER, método para combinar tokens que pertenecen a la misma entidad. Por ejemplo, "Buenos" y "Aires" se agrupan en una sola entidad "Buenos Aires".

**Context (Contexto)**  
En QA, el documento o párrafo que contiene la información necesaria para responder la pregunta. El modelo busca la respuesta dentro de este contexto.

---

## Recursos adicionales

- **Documentación oficial de Transformers:** https://huggingface.co/docs/transformers
- **Modelos en español:** https://huggingface.co/models?language=es
- **Curso gratuito de Hugging Face:** https://huggingface.co/course
- **Comunidad en español:** https://huggingface.co/spaces

---

*Este cuaderno fue diseñado con fines educativos para estudiantes de NLP en Argentina.*